<a href="https://colab.research.google.com/github/Piyu569/SparkProject01/blob/dev/SparkProject01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=8680d021c815ace5a5befad3a4a1a88853f5e3be5b621e05cb07cae8851df0e9
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [4]:
import pyspark

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName("banking_Pro").getOrCreate()

In [ ]:
#@title Credit-Card dataSet

In [9]:
df = spark.read.csv("/content/credit card.csv",inferSchema = True,header =True)

In [10]:
df.printSchema()

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



In [11]:
df.count()

10000

# This is the Distinct count of the members who are eligible for credit card

In [12]:
from pyspark.sql.functions import col
eligible = df.filter(df["CreditScore"] > 700).distinct().count()
print(eligible)

3116


# List of the Distinct Members

In [13]:
from pyspark.sql.functions import avg
list = df.select("CreditScore","Age","NumOfProducts").filter(df["CreditScore"]>700)
list.show()

+-----------+---+-------------+
|CreditScore|Age|NumOfProducts|
+-----------+---+-------------+
|        850| 43|            1|
|        822| 50|            2|
|        726| 24|            2|
|        732| 41|            2|
|        846| 38|            1|
|        756| 36|            1|
|        722| 29|            2|
|        804| 33|            1|
|        850| 36|            1|
|        834| 49|            1|
|        776| 32|            2|
|        829| 27|            1|
|        776| 37|            2|
|        788| 33|            2|
|        725| 19|            1|
|        742| 35|            1|
|        751| 36|            2|
|        735| 43|            2|
|        738| 58|            4|
|        813| 29|            1|
+-----------+---+-------------+
only showing top 20 rows



In [14]:
df.show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888| Mitchell|        850|    Spain|F

# List of members active and eligible for credit card

In [15]:
activeandeligible = df.filter((df["CreditScore"]>700).isNotNull() & (df["IsActiveMember"] == 1).isNotNull()).count()
print(activeandeligible)


10000


# Loan DataSet

In [16]:
df2 = spark.read.csv("/content/loan.csv",inferSchema = True,header = "True")

In [17]:
df2.show()
df2.printSchema()

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRIED|          6| 51000|      19999|            4|          SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|           

# Number of loans in each category

In [18]:
numberofloan = df2.groupBy("Loan Category").count().show()

+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|           HOUSING|   67|
|        TRAVELLING|   53|
|       BOOK STORES|    7|
|       AGRICULTURE|   12|
|         GOLD LOAN|   77|
|  EDUCATIONAL LOAN|   20|
|        AUTOMOBILE|   60|
|          BUSINESS|   24|
|COMPUTER SOFTWARES|   35|
|           DINNING|   14|
|          SHOPPING|   35|
|       RESTAURANTS|   41|
|       ELECTRONICS|   14|
|          BUILDING|    7|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
+------------------+-----+



# Number of people who have taken more than 1 lack loan

In [19]:
loanmorethanonelakh = df2.filter(df2["Loan Amount"]>"1,00,000").count()
print(loanmorethanonelakh)

379


# Number of people with income greater than 60000 rupees

In [20]:
incomemorethansistythousand = df2.filter(df2["Income"]>60000).count() 
print(incomemorethansistythousand)

198


#Number of people with 2 or more returned cheques and income less than 50000


In [21]:
df2.filter((df2[" Returned Cheque"]>"1") & (df2["Income"]<"50000")).count()


137

#Number of people with 2 or more returned cheques and are single

In [26]:
df2.filter((df2[" Returned Cheque"]>"1") & (df2["Marital Status"] =="SINGLE")).count()

111

# Number of people with expenditure over 50000 a month

In [28]:
df2.filter(df2["Expenditure"] > "50000").count()

6

# Txn DataSet

In [29]:
df3 =spark.read.csv("/content/txn.csv",inferSchema = True , header = True)

In [33]:
df3.show()
df3.printSchema()

+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|            null|    1000000.0|  1000000.0|
|409000611074'|TRF FROM  Indiafo...|  5-Jul-17|            null|    1000000.0|  2000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 18-Jul-17|            null|     500000.0|  2500000.0|
|409000611074'|TRF FRM  Indiafor...|  1-Aug-17|            null|    3000000.0|  5500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            null|     500000.0|  6000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            null|     500000.0|  6500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            null|     500000.0|  7000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            null|     500000.0|  7500000.0|

# COUNT OF TRANSACTION ON EVERY ACCOUNT

In [36]:
df3.groupBy("Account No").count().show()

+-------------+-----+
|   Account No|count|
+-------------+-----+
|409000438611'| 4588|
|     1196711'|10536|
|     1196428'|48779|
|409000493210'| 6014|
|409000611074'| 1093|
|409000425051'|  802|
|409000405747'|   51|
|409000362497'|29840|
|409000493201'| 1044|
|409000438620'|13454|
+-------------+-----+



# Maximum withdrawal amount

In [52]:
df3.groupBy("Account No").agg(max(" WITHDRAWAL AMT ").alias("maxamt")).orderBy("maxamt", ascending = False).show()

+-------------+-------------+
|   Account No|       maxamt|
+-------------+-------------+
|     1196711'|4.594475464E8|
|409000438620'|        4.0E8|
|409000425051'|       3.54E8|
|409000438611'|        2.4E8|
|409000405747'|        1.7E8|
|     1196428'|        1.5E8|
|409000362497'|1.413662392E8|
|409000493210'|        1.5E7|
|409000493201'|    2500000.0|
|409000611074'|     912000.0|
+-------------+-------------+



#MINIMUM WITHDRAWAL AMOUNT OF AN ACCOUNT


In [58]:
df3.groupBy("Account No").min(" WITHDRAWAL AMT ").orderBy("min( WITHDRAWAL AMT )").show()


+-------------+---------------------+
|   Account No|min( WITHDRAWAL AMT )|
+-------------+---------------------+
|409000493210'|                 0.01|
|409000438611'|                  0.2|
|     1196711'|                 0.25|
|     1196428'|                 0.25|
|409000438620'|                 0.34|
|409000362497'|                 0.97|
|409000425051'|                 1.25|
|409000493201'|                  2.1|
|409000405747'|                 21.0|
|409000611074'|                120.0|
+-------------+---------------------+



#MAXIMUM DEPOSIT AMOUNT OF AN ACCOUNT


In [60]:
df3.groupBy("Account No").max(" DEPOSIT AMT ").orderBy("max( DEPOSIT AMT )",ascending = False).show()


+-------------+------------------+
|   Account No|max( DEPOSIT AMT )|
+-------------+------------------+
|409000438620'|           5.448E8|
|     1196711'|             5.0E8|
|     1196428'|     2.119594422E8|
|409000405747'|           2.021E8|
|409000362497'|             2.0E8|
|409000438611'|          1.7025E8|
|409000425051'|             1.5E7|
|409000493210'|             1.5E7|
|409000611074'|         3000000.0|
|409000493201'|         1000000.0|
+-------------+------------------+



#MINIMUM DEPOSIT AMOUNT OF AN ACCOUNT

In [61]:
df3.groupBy("Account No").min(" DEPOSIT AMT ").orderBy("min( DEPOSIT AMT )").show()


+-------------+------------------+
|   Account No|min( DEPOSIT AMT )|
+-------------+------------------+
|409000493210'|              0.01|
|409000362497'|              0.03|
|409000438611'|              0.03|
|409000438620'|              0.07|
|409000493201'|               0.9|
|409000425051'|               1.0|
|     1196428'|               1.0|
|     1196711'|              1.01|
|409000405747'|             500.0|
|409000611074'|            1320.0|
+-------------+------------------+



#sum of balance in every bank account


In [63]:
df3.groupBy("Account No").sum("BALANCE AMT").show()


+-------------+--------------------+
|   Account No|    sum(BALANCE AMT)|
+-------------+--------------------+
|409000438611'|-2.49486577068339...|
|     1196711'|-1.60476498101275E13|
|     1196428'| -8.1418498130721E13|
|409000493210'|-3.27584952132095...|
|409000611074'|       1.615533622E9|
|409000425051'|-3.77211841164998...|
|409000405747'|-2.43108047067000...|
|409000362497'| -5.2860004792808E13|
|409000493201'|1.0420831829499985E9|
|409000438620'|-7.12291867951358...|
+-------------+--------------------+



#Number of transaction on each date


In [67]:
df3.groupBy("VALUE DATE").count().orderBy(col("count").desc()).show()

+----------+-----+
|VALUE DATE|count|
+----------+-----+
| 27-Jul-17|  567|
| 13-Aug-18|  463|
|  8-Nov-17|  402|
|  7-Oct-17|  382|
| 10-Jul-18|  374|
| 12-Dec-17|  367|
| 12-Sep-18|  365|
|  9-Aug-18|  360|
| 19-Sep-17|  358|
| 16-Mar-17|  353|
| 10-Sep-18|  344|
| 14-Jul-17|  333|
|  7-Mar-18|  319|
| 11-Oct-18|  303|
| 22-Aug-17|  301|
|  9-Jan-18|  299|
|  9-Oct-18|  297|
| 20-Apr-18|  296|
|  9-Jul-18|  292|
|  7-Apr-18|  291|
+----------+-----+
only showing top 20 rows



#List of customers with withdrawal amount more than 1 lakh


In [71]:
df3.filter((df3[" WITHDRAWAL AMT "]>"100000.0").isNotNull()).show()

+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        133900.0|         null|  8366100.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|         18000.0|         null|  8348100.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|          5000.0|         null|  8343100.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        195800.0|         null|  8147300.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|         81600.0|         null|  8065700.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|         41800.0|         null|  8023900.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|         98500.0|         null|  7925400.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        143800.0|         null|  7781600.0|